<a href="https://colab.research.google.com/github/victoriaazabel/SEdigitalorganizations/blob/main/VictoriaZabel_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following assignment consists of a theoretical part (learning portfolio) and a practical part (assignment). The goal is to build a classification model that predicts from which subject area a certain abstract originates. The plan would be that next week we will discuss your learnings from the theory part, that means you are relatively free to fill your Learning Portfolio on this new topic and in two weeks we will discuss your solutions of the Classification Model.

#Theory part (filling your Learning Portfolio, May 10)

In preparation for the practical part, I ask you to familiarize yourself with the following resources in the next week:

1) Please watch the following video:

https://course.fast.ai/Lessons/lesson4.html

You are also welcome to watch the accompanying Kaggle notebook if you like the video.

2) In addition to the video, I recommend you to read the first chapters of the course

https://huggingface.co/learn/nlp-course/chapter1/1


Try to understand principle processes and log them in your learning portfolio! A few suggestions: What is a pre-trained NLP model? How do I load them? What is tokenization? What does fine-tuning mean? What types of NLP Models are there? What possibilities do I have with the Transformers package? etc...

#Practical part (Assignment, May 17)

1) Preprocessing: The data which I provide as zip in Olat must be processed first, that means we need a table which has the following form:

Keywords | Title | Abstract | Research Field

The research field is determined by the name of the file.

2) We need a training dataset and a test dataset. My suggestion would be that for each research field we use the first 5700 lines for the training dataset and the last 300 lines for the test dataset. Please stick to this because then we can compare our models better!

3) Please use a pre-trained model from huggingface to build a classification model that tries to predict the correct research field from the 26. Please calculate the accuracy and the overall accuracy for all research fields. If you solve this task in a group, you can also try different pre-trained models. In addition to the abstracts, you can also see if the model improves if you include keywords and titles.

Some links, which can help you:

https://huggingface.co/docs/transformers/training

https://huggingface.co/docs/transformers/tasks/sequence_classification

One last request: Please always use PyTorch and not TensorFlow!

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
#!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
"""
1. [{'generated_text': 'This summer, I am going to the gym, and when I see what works, I have already finished my morning routine. I have found myself feeling really good. And the body is perfect. I'},
 {'generated_text': 'This summer, I am going to see what they think of a new album.”\nBut then, I get to see how the process works with the other two albums, one of which is'}]
2. [{'generated_text': 'This summer, I am going to share the latest in a long list of new information about the company, with more on the company today, particularly concerning its upcoming fiscal 2015. What follows are the most'},
 {'generated_text': 'This summer, I am going to try and get a new one out. I will look at my current profile as the most prominent female journalist by name, and whether she is a good editor, or'}]
3. [{'generated_text': "This summer, I am going to be playing a lot more.\n\nThis spring, I joined a handful of indie indie bands. I'm now a fan of indie music that includes indie pop, punk, and indie rock. In general, I"},
 {'generated_text': 'This summer, I am going to try and learn the tricks here at The Cauldron as I work on making a lot of fun and interesting stuff so I have to make it a reality.\nI have a lot of fun writing new stories. First'}]
"""

"""
generate = pipeline("text-generation", model="distilgpt2")
generate(
    "This summer, I am going to",
    max_length=50,
    num_return_sequences=2,
)
"""

"""
From pipeline 2:
1. [{'generated_text': "This summer, I am going to Greece. I will go to Colombian and other European countries. I will be going to Spain if I am not in the country. I would say in Spain that I'd be staying for the better part of the year"},
 {'generated_text': "This summer, I am going to Greece. I will have a whole year. It's a big deal. It's an important moment for me. I want to tell everyone that it is a great country. I want to tell people what it is"}]
2. [{'generated_text': "This summer, I am going to Greece. I will do this, but I don't want to talk about the risks of getting there. I want the Greeks to come. What is my hope, though? What is my hope then? Is there"},
 {'generated_text': 'This summer, I am going to Greece. I will put two big boxes on a table with three of my students. The first one is theindependence party, and the second comes from the party called The Nationalists and Democracy, one of four I'}]
"""

generate2 = pipeline("text-generation", model="distilgpt2")
generate2(
    "This summer, I am going to Greece. I will",
    max_length=50,
    num_return_sequences=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "This summer, I am going to Greece. I will do this, but I don't want to talk about the risks of getting there. I want the Greeks to come. What is my hope, though? What is my hope then? Is there"},
 {'generated_text': 'This summer, I am going to Greece. I will put two big boxes on a table with three of my students. The first one is theindependence party, and the second comes from the party called The Nationalists and Democracy, one of four I'}]

In [ ]:
#2. Mask Filling

"""
1. "This summer I will be visiting many <mask> islands."
[{'score': 0.21606504917144775,
  'token': 7599,
  'token_str': ' Caribbean',
  'sequence': 'This summer I will be visiting many Caribbean islands.'},
 {'score': 0.20062261819839478,
  'token': 19806,
  'token_str': ' Hawaiian',
  'sequence': 'This summer I will be visiting many Hawaiian islands.'}]

2. "We need to reconsider the <mask> of our trip this summer."
[{'score': 0.175999715924263,
  'token': 5801,
  'token_str': ' timing',
  'sequence': 'We need to reconsider the timing of our trip this summer.'},
 {'score': 0.11877398192882538,
  'token': 2698,
  'token_str': ' direction',
  'sequence': 'We need to reconsider the direction of our trip this summer.'}]

3. "Have you <mask> the professor about the scope of our work yey?"
[{'score': 0.3536241948604584,
  'token': 553,
  'token_str': ' asked',
  'sequence': 'Have you asked the professor about the scope of our work yet?'},
 {'score': 0.19492030143737793,
  'token': 18998,
  'token_str': ' consulted',
  'sequence': 'Have you consulted the professor about the scope of our work yet?'}]
"""

from transformers import pipeline

unmask = pipeline("fill-mask")
unmask("Have you <mask> the professor about the scope of our work yet?", top_k=2)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'score': 0.3536241948604584,
  'token': 553,
  'token_str': ' asked',
  'sequence': 'Have you asked the professor about the scope of our work yet?'},
 {'score': 0.19492030143737793,
  'token': 18998,
  'token_str': ' consulted',
  'sequence': 'Have you consulted the professor about the scope of our work yet?'}]

In [ ]:
# 3. NER (Named Entity Recognition)

from transformers import pipeline

"""
1. "Sup, Ladies, my name is Slim Shady, I'm the lead singer of D12, baby."
[{'entity_group': 'PER',
  'score': 0.9916236,
  'word': 'Slim Shady',
  'start': 24,
  'end': 34},
 {'entity_group': 'ORG',
  'score': 0.99210477,
  'word': 'D12',
  'start': 59,
  'end': 62}]

"""

ner1 = pipeline("ner", grouped_entities=True)
ner1("Sup, Ladies, my name is Slim Shady, I'm the lead singer of D12, baby.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.9916236,
  'word': 'Slim Shady',
  'start': 24,
  'end': 34},
 {'entity_group': 'ORG',
  'score': 0.99210477,
  'word': 'D12',
  'start': 59,
  'end': 62}]

In [ ]:
# 4. Question Answering

"""
1.  question="What are we doing today?",
    context="The weather is nice so we're gonna go swimming at the beach"
{'score': 0.32059288024902344,
 'start': 35,
 'end': 59,
 'answer': 'go swimming at the beach'}
2. question="What are we having for dinner?",
    context="I don't know, but I'm really in the mood for sushi."
{'score': 0.9471299648284912, 'start': 45, 'end': 50, 'answer': 'sushi'}
3. question="Who is your favorite music artist?",
    context="Out of all musical artists that I have been listening to, including Taylor Swift, Eminem is by far my favorite."
{'score': 0.7994513511657715, 'start': 82, 'end': 88, 'answer': 'Eminem'}
"""

from transformers import pipeline

question_answer = pipeline("question-answering")
question_answer(
    question="What are we doing today?",
    context="The weather is nice so we're gonna go swimming at the beach.",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.32059288024902344,
 'start': 35,
 'end': 59,
 'answer': 'go swimming at the beach'}

In [ ]:
# 5. Text Summarization

"""
1. Two paragraphs of the 26 best islands in Greece, 3 were too many
[{'summary_text': " Santorini's volcanic eruption blew out its heart 3,500 years ago, creating fertile ground for exceptional Assyrtiko grapes and Vinsanto wines . Syros, capital of the Cyclades, is built on twin peaks – one Orthodox, the other Catholic . The colourful 19th-century city of Ermoupoli’s marble piazzas, princely mansions, and miniature replica of La Scala ."}]
"""

from transformers import pipeline

summarize = pipeline("summarization")
summarize("""The best Greek islands to visit in 2023

1. Santorini
Best for: Honeymooners and first-timers
Cooing American and Chinese honeymooners line up to take selfies as the sun sinks behind Santorini's caldera, the flooded volcanic crater. That view may be a romantic cliché, but it still takes your breath away. A volcanic explosion blew out Santorini's heart 3,500 years ago, leaving black-sand beaches, vertiginous cliffs in psychedelic hues, and swirling rumours about Atlantis in its wake. The eruption also preserved the ancient city of Akrotiri under layers of ash, and created fertile ground for exceptional Assyrtiko grapes and Vinsanto wines. (Sample them at Domaine Sigalas and Vassaltis wineries, paired with delicate dishes that let the grapes sing.)
Apart from a boat trip to the smouldering crater of Nea Kameni and hot springs at Palia Kameni, there's not much to do but gaze at the mesmerising views from your suite, dangling on the edge of the caldera. Most places to stay are concentrated in Oia and Imerovigli, but the inland village of Pyrgos is up-and-coming. Go for a twilight Bellini at Franco's Cafe and visit Emporio, with its smattering of old-school coffee shops and Airbnbs. For a glimpse of Santorini before the onslaught of cruise ships and Instagrammers, explore the quieter south (but keep your discoveries to yourself).

2. Syros
Best Greek island for: Culture and off-season cachet
On Syros, capital of the Cyclades, you won’t find sugar-cube villages and whitewashed lanes. The colourful 19th-century city of Ermoupoli is built on twin peaks – one Orthodox, the other Catholic, the heritage of a long Venetian occupation. There’s still a strong Italian flavour in Ermoupoli’s marble piazzas, princely mansions, and miniature replica of La Scala, the showpiece of a year-round cultural scene. Syros hosts festivals of animation, dance, digital art, film, classical music, jazz and rembetiko, the Greek blues popularised by local musician Markos Vamvakaris. A few rembetiko joints have survived in the upper town, Ano Syros.
Once Greece’s ship-building centre, Syros' industry centres around the yard in Neorio. But the most splendid legacy of the shipping industry are the manor houses in Vaporia and Poseidonia. The beaches are slightly less splendid — with the exception of Delfini, Varvarousa, and Aetos in the wild north. But fabulous seaside tavernas abound: Iliovassilemar on Galissas beach for samphire and sea-urchin salad and rockfish soup; Allou Yialou in the pretty seaside village of Kini for lobster with orzo. In Ermoupoli, the finest places to eat and drink are around Androu Street: Ousyra, where the chef plates up Greek-ified pasta and beautifully balanced salads, and Django Gelato, where the pistachio gelato reigns supreme, and the fig sorbet made in August can sell out in less than half an hour. Perhaps the prettiest restaurant of all is Mazi, a vine-covered courtyard festooned with bougainvillaea. Before you leave, stock up on loukoumi (rose-tinted Turkish delight) and San Michali cheese from Prekas delicatessen, and visit Zeyelo for hand-made wooden sunglasses. For more recommendations, see our insider guide to Syros.
""")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': " Santorini's volcanic eruption blew out its heart 3,500 years ago, creating fertile ground for exceptional Assyrtiko grapes and Vinsanto wines . Syros, capital of the Cyclades, is built on twin peaks – one Orthodox, the other Catholic . The colourful 19th-century city of Ermoupoli’s marble piazzas, princely mansions, and miniature replica of La Scala ."}]

In [3]:
#6. Translation
from transformers import pipeline

#[{'translation_text': 'I want to go to Milos this summer.'}]

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Je veux aller à Milos cet été.")

[{'translation_text': 'I want to go to Milos this summer.'}]

In [4]:
#7. Sentiment analysis
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I can't wait to go on vacation in July.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9967976212501526}]

In [5]:
# 8. Zero Shot Classification
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This song discusses the rapper's struggle with drugs",
    candidate_labels=["music", "substance abuse", "rappers"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': "This song discusses the rapper's struggle with drugs",
 'labels': ['substance abuse', 'music', 'rappers'],
 'scores': [0.5511703491210938, 0.3627069294452667, 0.08612275123596191]}

In [ ]:
# Tokenization: splitting by spacing 
# output: ['I', 'wanna', 'go', 'on', 'vacation']
tokenized = "I wanna go on vacation".split()
print(tokenized)


['I', 'wanna', 'go', 'on', 'vacation']


In [ ]:
#YOUR TASK

Addition: Accuracy measures whether the research field with the highest probability value matches the target. With 26 research fields, it would also be interesting to know if the correct target is at least among the three highest probability values.

$\begin{pmatrix} A\\ B \\ C \\D \\E \end{pmatrix} = \begin{pmatrix} 0.1\\ 0.95 \\ 0.5 \\0.2 \\0.3 \end{pmatrix} → \text{Choice}_1 = B, \text{Choice}_3 = B,C,E$